## Aprendizaje de reglas de asociación

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [93]:
df_ratings = pd.read_csv('./ml-20m/ratings.csv')
df_movies = pd.read_csv('./ml-20m/movies.csv')

In [94]:
display(df_ratings.head())
display(df_movies.head())

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [95]:
df_ratings = df_ratings[df_ratings['rating']>3]
#df_ratings = df_ratings.sample(df_ratings.shape[0]//10)

In [107]:
df = df_ratings.merge(df_movies,left_on='movieId', right_on='movieId', how= "inner")
display(df.head())
df= df.sort_values( by='userId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
df=df.values[:,[0,4]]
display(df)

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,91,2,3.5,1112061358,Jumanji (1995),Adventure|Children|Fantasy
2,119,2,4.0,845110667,Jumanji (1995),Adventure|Children|Fantasy
3,142,2,4.0,833458658,Jumanji (1995),Adventure|Children|Fantasy
4,156,2,5.0,1040937649,Jumanji (1995),Adventure|Children|Fantasy


array([[1, 'Jumanji (1995)'],
       [1, 'Untouchables, The (1987)'],
       [1, 'Willow (1988)'],
       ...,
       [138493, 'Starship Troopers (1997)'],
       [138493, 'Man on the Moon (1999)'],
       [138493, 'Pinocchio (1940)']], dtype=object)

In [108]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [109]:
print('Movies -- dimensions: {0};   size: {1}'.format(df.shape, size(df)))

Movies -- dimensions: (12195566, 2);   size: 0.00 MB


In [110]:
transactions=[]
for user_id, movie in groupby(df, lambda x: x[0]):
    transactions.append([item[1] for item in movie])
print(transactions)
#con toda la base..
#IOPub data rate exceeded.
#The notebook server will temporarily stop sending output

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [120]:
from efficient_apriori import apriori
#wARNING-CUIDADO!! no poner min_support pequeño como por ejempo 0.001!!
#min_support y min_confiden entre 0 y 1  
itemsets, rules = apriori(transactions, min_support=0.2,  min_confidence=0.5)

In [121]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules=sorted(rules, key=lambda rule: rule.confidence)
for rule in rules:
  print(rule) # Prints the rule and its confidence, support, lift, ...

{Shawshank Redemption, The (1994)} -> {Usual Suspects, The (1995)} (conf: 0.509, supp: 0.214, lift: 1.674, conv: 1.418)
{Shawshank Redemption, The (1994)} -> {Schindler's List (1993)} (conf: 0.512, supp: 0.215, lift: 1.607, conv: 1.396)
{Pulp Fiction (1994)} -> {Usual Suspects, The (1995)} (conf: 0.537, supp: 0.214, lift: 1.763, conv: 1.501)
{Pulp Fiction (1994)} -> {Forrest Gump (1994)} (conf: 0.541, supp: 0.216, lift: 1.448, conv: 1.365)
{Shawshank Redemption, The (1994)} -> {Forrest Gump (1994)} (conf: 0.550, supp: 0.231, lift: 1.471, conv: 1.391)
{Silence of the Lambs, The (1991)} -> {Forrest Gump (1994)} (conf: 0.564, supp: 0.220, lift: 1.508, conv: 1.435)
{Forrest Gump (1994)} -> {Pulp Fiction (1994)} (conf: 0.578, supp: 0.216, lift: 1.448, conv: 1.424)
{Forrest Gump (1994)} -> {Silence of the Lambs, The (1991)} (conf: 0.589, supp: 0.220, lift: 1.508, conv: 1.483)
{Shawshank Redemption, The (1994)} -> {Silence of the Lambs, The (1991)} (conf: 0.600, supp: 0.252, lift: 1.537, conv

# Informe
Inicialmente, y a modo de reducción del dataset, se obtuvieron únicamente las películas vistas con un rating mayor a 2, con esto logramos que el procesamiento sea posible con un hardware limitado.
Las reglas de asociación se obtuvieron limitando el soporte mínimo a 0.2, es decir, la película debe aparecer al menos en un 20% de las transacciones utilizadas. 
Todas las asociaciones encontradas tienen una confidencia mayor a 0.5 (parámetro establecido como métrica), indicando que se observa la misma en al menos en el 50% de las transacciones.
En todos los casos, el lift supera a 1, esto nos da un indicador del grado de dependencia entre el antecedente y el consecuente, mientras mayor sea, mayor será la dependencia. Inclusive podemos observar que a mayor grado de confidencia, mayor grado del lift.
Finalmente, la convicción elevada nos da el grado de ocurrencias del presedente sin el consecuente mostrado, esto puede darse  o bien porque el presedente tiene muchas otras relaciones que se dan con un soporte bajo.